# MIMIC-FHIR Medication
An exploration of the medication in MIMIC and how best to map into FHIR

In [ ]:
%matplotlib inline
# Imports
import pandas as pd
import numpy as np
import psycopg2
from pathlib import Path
import json
from uuid import uuid5, NAMESPACE_OID
import datetime

# Ubuntu backend not set up properly, switch to nbAgg
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
plt.switch_backend('nbAgg')
plt.get_backend()

plt.style.use('ggplot')
plt.rcParams.update({'font.size': 20})

In [ ]:
# Database connection
sqluser = 'postgres'
sqlpass = 'postgres'
dbname = 'mimic'
host = 'localhost'

db_conn = psycopg2.connect(dbname=dbname, user=sqluser, password=sqlpass, host=host)

# Medication in FHIR
Medication use in a hospital follows the flow of *prescribe -> dispense -> administer*. 

The FHIR representation of that flow is *MedicationRequest -> MedicationDispense -> MedicationAdministration*. 

To translate MIMIC medication information into FHIR we need to map the MIMIC tables into these three medication event resources plus a medication resource:
- Medication: The definition of a medication or medication mix
- MedicationRequest: The order for a medication and base instructions for use
- MedicationDispense: The dispense event for a medication 
- MedicationAdministration: The administration event for a medication to a patient

The primary medication tables in MIMIC are:
- *prescriptions*: Provides information about prescribed medication
- *pharmacy*: Provides information on filled medication based on a prescription
- *emar*/*emar_detail*: The record of a medication administration to a patient
  - emar events are only available for about half the patients as the system did not come into affect till post 2015
- *inputevents*: ICU medication administration to a patient

Throughout this tutorial you accomplish the following:
- Differentiate how MIMIC and FHIR store medication
- Follow a full patient medication workflow
- See fragmented medication data (the reality of data)





# <u> MIMIC to FHIR medication translation </u>
The medication mapped from MIMIC to FHIR are best described with some examples. The examples will use the following variables:

In [ ]:
# Medication UUID namespaces
ns_mimic = uuid5(NAMESPACE_OID, 'MIMIC-IV')
ns_med = uuid5(ns_mimic, 'Medication')
ns_medreq = uuid5(ns_mimic, 'MedicationRequest')
ns_meddisp = uuid5(ns_mimic, 'MedicationDispense')
ns_medadmin = uuid5(ns_mimic, 'MedicationAdministration')
ns_patient = uuid5(ns_mimic, 'Patient')

In [ ]:
# Medication
single_med_uuid = '8ef6df9c-e5bc-5c98-bc81-765f9f04bd81' # uuid generated from medication name
med_mix_uuid = '7d960cd4-cef5-521d-9b77-6b6c86ce2b56' # uuid generated from medication names concatentated

# MedicationRequest
single_medreq_id = 50814
single_medreq_uuid = str(uuid5(ns_medreq,str(single_medreq_id)))
medreq_mix_id = 15849072
medreq_mix_uuid = str(uuid5(ns_medreq,str(medreq_mix_id)))
poe_medreq_id = '10000032-126'
poe_medreq_uuid = str(uuid5(ns_medreq,str(poe_medreq_id)))

#MedicationDispense
meddisp_id = 26267497
meddisp_uuid = str(uuid5(ns_meddisp,str(meddisp_id)))

#MedicationAdministration
medadmin_id = '10002443-26-1.10' # combination of emar_id, and parent_field_ordinal
medadmin_uuid = str(uuid5(ns_medadmin,str(medadmin_id)))

In [ ]:
medadmin_uuid

In [ ]:
str(a)

## 1. Creating Medication resources from MIMIC
Medication is not stored in one data table in MIMIC but distributed across multiple. The medication sources from MIMIC are:
- mimic_hosp.prescriptions drug
- mimic_icu.d_items label
  - d_items contains a subset of the total medication in MIMIC, as these are the most applicable to the ICU

The tables are all combined together and distinct medication are kept.There are two main types of medication resources created. 
1. Single medication 
2. Medication mix (multiple)

### 1.1 Medication: Single Medication
Prescriptions with only one drug associated with them fall in this category (majority of prescriptions). All d_items medication are single medication resources. For this example we will pull in a prescription medication and compare it to the FHIR Medication resource.

In [ ]:
q_single_med = f""" 
    SELECT *
    FROM mimic_hosp.prescriptions pr
    WHERE pr.pharmacy_id = {single_medreq_id}
"""
single_med = pd.read_sql_query(q_single_med, db_conn)
single_med.head()

And in FHIR the same medication information looks like this:

In [ ]:
q_single_med_fhir = f""" 
    SELECT * 
    FROM mimic_fhir.medication
    WHERE id = '{single_med_uuid}' -- uuid generated from pharmacy_id
"""
single_med_fhir = pd.read_sql_query(q_single_med_fhir, db_conn)
print(json.dumps(single_med_fhir.fhir[0], indent=4))

### 1.2 Medication: Medication Mix
For some prescriptions there are multiple medication associated with a single pharmacy_id. In FHIR medication request and administrations can only reference a single medication, so these medication mixes must be made into a single Medication resource.

In [ ]:
q_med_mix = f""" 
    SELECT *
    FROM mimic_hosp.prescriptions 
    WHERE pharmacy_id = {medreq_mix_id} 
"""
med_mix = pd.read_sql_query(q_med_mix, db_conn)
med_mix.head()

And in FHIR the same information is reprented in a json format. There are are two main things to notice here:
- The medication code is a concatentation of all the drug names. The format for the concatentation is based on the `drug_type` in prescriptions. So the format is effectively MAIN_BASE_ADDITIVE_ADDITIVE, where the additives are ordered alphabetically.
- The second is that in a medicaiton mix, the base medication are all referenced in ingredients. If you looked up each ingredient you would get a Medication resource for the MAIN, BASE, and ADDITIVE medication.

In [ ]:
q_med_mix_fhir = f""" 
    SELECT * 
    FROM mimic_fhir.medication
    WHERE id = '{med_mix_uuid}' -- uuid generated from pharmacy_id
"""
med_mix_fhir = pd.read_sql_query(q_med_mix_fhir, db_conn)
print(json.dumps(med_mix_fhir.fhir[0], indent=4))

## 2. Creating MedicationRequest from MIMIC
The prescriptions table holds the primary medication request information in MIMIC. Secondary information is pulled in from *pharmacy* and some IV med requests are created separtely from *poe* when information is not in *prescriptions*. There are three main types of medication request that are created:
1. Single medication prescription
2. Medication mixes representing multiple medications requested in one prescription
3. IV medication that are not in *prescriptions*. This is where *poe* comes in

### 2.1. MedicationRequest: Single Medication
- To get a single medication prescription group by the pharmacy_id and only grab prescriptions with one drug linked
- For this example that has already been completed so one prescription is selected with a single medication
- The pharmacy table has been joined to grab the `entertime` of the prescription, since it is useful information to have for the medication request

In [ ]:
q_single_med = f""" 
    SELECT 
        pr.*
        , ph.entertime
        , ph.medication
    FROM 
        mimic_hosp.prescriptions pr
        LEFT JOIN mimic_hosp.pharmacy ph
            ON pr.pharmacy_id = ph.pharmacy_id
    WHERE pr.pharmacy_id = {single_medreq_id}
"""
single_med = pd.read_sql_query(q_single_med, db_conn)
single_med.head()

In FHIR this same row will be represented as a MedicationRequest. There are a few things to note here:
- `id` is a generated uuid based on `pharmacy_id`
- The `pharmacy_id` is still kept in its raw form but stored as an `identfier.value`
- All `status` are set to 'completed' and `intent` to 'order' for MIMIC prescriptions
- `product_strength` is stored in `dosageInstruction.text` so that general information can get into the resource
- The MedicationRequest has references out to Patient, Encounter, and Medication

In [ ]:
q_single_med_fhir = f""" 
    SELECT * 
    FROM mimic_fhir.medication_request 
    WHERE id = '{single_medreq_uuid}' -- uuid generated from pharmacy_id
"""
single_med_fhir = pd.read_sql_query(q_single_med_fhir, db_conn)
print(json.dumps(single_med_fhir.fhir[0], indent=4))

### 2.2 MedicationRequest: Medication Mix
- To get a medication mix prescription, group by the pharmacy_id and only grab prescriptions with 2+ drugs linked
- For this example that has already been completed, so one prescription is selected with a medication mix

In [ ]:
q_med_mix = f""" 
    SELECT 
        pr.*
        , ph.entertime
    FROM 
        mimic_hosp.prescriptions pr
        LEFT JOIN mimic_hosp.pharmacy ph
            ON pr.pharmacy_id = ph.pharmacy_id
    WHERE pr.pharmacy_id = {medreq_mix_id}
"""
med_mix = pd.read_sql_query(q_med_mix, db_conn)
med_mix.head()

In FHIR all four rows will be collapsed into one MedicationRequest. Things to note:
- The resource is effectively the same as the single medication request
- There is only one Medication referenced in MedicationRequest, even throuhg this prescription has for meds. The medication have been grouped into a medication mix as described in the [Medication mix section](#1.2-Medication:-Medication-Mix). 
- Additional information is pulled in from *pharmacy* to get the `repeat.duration`

In [ ]:
q_med_mix_fhir = f""" 
    SELECT * 
    FROM mimic_fhir.medication_request 
    WHERE id = '{medreq_mix_uuid}' -- uuid generated from pharmacy_id
"""
med_mix_fhir = pd.read_sql_query(q_med_mix_fhir, db_conn)
print(json.dumps(med_mix_fhir.fhir[0], indent=4))

### 2.3 MedicationRequest: poe orders
For common medication used in the hospital there are times when no prescription is created. These orders are still present in *poe*. All medication requests that do not show up in *prescriptions* or *pharmacy* are present in *poe*. There is less detailed information available for these orders but they are often straight IV meds.

In [ ]:
q_poe_medreq = f""" 
    SELECT
        poe.poe_id 
        , poe.subject_id 
        , poe.hadm_id 
        , poe.ordertime 
        , order_status
        , em.medication

    FROM
        mimic_hosp.poe poe
        LEFT JOIN mimic_hosp.emar em
            ON poe.poe_id = em.poe_id 
    WHERE poe.poe_id = '{poe_medreq_id}'
"""
poe_medreq = pd.read_sql_query(q_poe_medreq, db_conn)
poe_medreq.head()

In FHIR the poe order creates a MedicationRequest that will be smaller than the *prescriptions* MedicationRequest because there is less supporting information. The essential information is still present:
- References to Patient, Encounter, and Medication
- The `status` is set, but most poe order status are NULL with only a few being active/inactive
- The `authoredOn` time is present, as it is useful to know when the order was created

In [ ]:
q_poe_medreq_fhir = f""" 
    SELECT * 
    FROM mimic_fhir.medication_request 
    WHERE id = '{poe_medreq_uuid}' -- uuid generated from poe_id
"""
poe_medreq_fhir = pd.read_sql_query(q_poe_medreq_fhir, db_conn)
print(json.dumps(poe_medreq_fhir.fhir[0], indent=4))

## 3. Creating MedicationDispense from MIMIC
In MIMIC not all medication workflows include a dispense step. The source of dispense is *pharmacy*. So for ICU inputevents there is no dispense event so the flow just become MedicationRequest->MedicationAdministration.

The MedicationDispense resource takes information from the prescription as well as reference the medication administrations that use the dispense. The *pharmacy* table holds the dispense information:

In [ ]:
q_meddisp = f""" 
    SELECT *
    FROM
        mimic_hosp.pharmacy
    WHERE pharmacy_id = '{meddisp_id}'
"""
meddisp = pd.read_sql_query(q_meddisp, db_conn)
meddisp.head()

The MedicationDisepense is a good encapsulation of the medication workflow. What we see:
- The dispense references the Medication dispensed, the MedicationRequest that lead to the dispense and the MedicationAdministrations that used the dispensed medication
  - Multiple MedicationAdministration can be referenced
- `dosageInstruction` come from the *pharmacy* details
- `status` is fixed to 'completed' as there is no valid mapping for a dispense status in MIMIC

In [ ]:
q_meddisp_fhir = f""" 
    SELECT * 
    FROM mimic_fhir.medication_dispense
    WHERE id = '{meddisp_uuid}' -- uuid generated from pharmacy_id
"""
meddisp_fhir = pd.read_sql_query(q_meddisp_fhir, db_conn)
print(json.dumps(meddisp_fhir.fhir[0], indent=4))

## 4. Creating MedicationAdministration from MIMIC
Medication administration is split between two locations in MIMIC: *emar* and *inputevents*. For now this tutorial will focus on *emar*. *emar* records every medication administration with barcode reads. The *emar* table links with the *emar_detail* table to get specifics if there were multiple steps in administering the medicaiton. 


A combo of *emar* and *emar_detail* are used to generate MedicationAdministration. Each row in *emar_detail* is taken as a MedicationAdministration with info from *emar* filling in the blanks.



In [ ]:
q_meddisp = f""" 
    SELECT 
        emd.emar_id
        , parent_field_ordinal
        , emd.pharmacy_id
        , dose_given
        , dose_given_unit
        , product_description
        , em.medication
    FROM
        mimic_hosp.emar_detail emd
        LEFT JOIN mimic_hosp.emar em
            ON emd.emar_id = em.emar_id
    WHERE emd.emar_id || '-' || parent_field_ordinal = '{medadmin_id}'
"""
meddisp = pd.read_sql_query(q_meddisp, db_conn)
meddisp.head()

The MedicationAdministration resource will grab the above to create the resource. Some notes:
- The administration will reference the Medication, MedicationRequest, Patient and Encounter
- The administration does NOT reference the MedicationDispense (yet, no spot in the resource currently)
- The actual dose given is supplied in `dosage.dose`
- The time of medication administration is stored in `effectiveDateTime`
- If you want to see the actual medication, you need to pull in the referenced Medication resource

In [ ]:
q_medadmin_fhir = f""" 
    SELECT * 
    FROM mimic_fhir.medication_administration
    WHERE id = '{medadmin_uuid}' -- uuid generated from emar_id and parent_field_ordinal
"""
medadmin_fhir = pd.read_sql_query(q_medadmin_fhir, db_conn)
print(json.dumps(medadmin_fhir.fhir[0], indent=4))

Pull in Medication resource based on the medicationReference. This will show what medication is actually administered

In [ ]:
med_uuid = medadmin_fhir.fhir[0]['medicationReference']['reference'].split('/')[1]

q_medadmin_med = f""" 
    SELECT * 
    FROM mimic_fhir.medication
    WHERE id = '{med_uuid}' 
"""
medadmin_med = pd.read_sql_query(q_medadmin_med, db_conn)
print(json.dumps(medadmin_med.fhir[0], indent=4))

## 5. FHIR medication following patients 
We can learn a lot of how the FHIR medication works by following a couple patients medication experiences.

We will look at 3 cases
1. Patient with emar information
2. Patient without emar information
3. Patient in the ICU, so using inputevents

What we will have
- A couple medication request
- A couple medication dispense
- Many emar events
- Show linking back to medication
- Some visual? 
  - Have timing for the request
  - Have timing for the administration

### 5.1 Patient with emar information

In [ ]:
# Medication details
subject_id = 10002618
patient_uuid = str(uuid5(ns_patient, str(subject_id)))

In [ ]:
# Get medrequests associated
q_medreq_p1 = f""" 
    SELECT * 
    FROM mimic_fhir.medication_request
    WHERE patient_id = '{patient_uuid}' 
"""
medreq_p1 = pd.read_sql_query(q_medreq_p1, db_conn)

# Get meddispenses associated
q_meddisp_p1 = f""" 
    SELECT * 
    FROM mimic_fhir.medication_dispense
    WHERE patient_id = '{patient_uuid}' 
"""
meddisp_p1 = pd.read_sql_query(q_meddisp_p1, db_conn)

# Get medadmin associated
q_medadmin_p1 = f""" 
    SELECT * 
    FROM mimic_fhir.medication_administration
    WHERE patient_id = '{patient_uuid}' 
"""
medadmin_p1 = pd.read_sql_query(q_medadmin_p1, db_conn)

print(f'Count of references: MedicationRequest: {medreq_p1.shape[0]}, MedicationDispense: {meddisp_p1.shape[0]}, MedicationAdministration: {medadmin_p1.shape[0]}')

From the patient we pulled in we can start to see the medication workflow. Medication gets prescribed with MedicationRequests, then dispensed from the pharmacy with MedicationDispense, and finally the medication gets administred in MedicationAdministration.

But then why are there more requests than dispenses? This arises from multiple origins of MedicationRequests. We can look at the origin based on the identifier.type in MedicationRequest

In [ ]:
id_type = [fhir['identifier'][0]['type']['coding'][0]['value'] for fhir in medreq_p1.fhir]
print(f'Pharmacy ID occur {id_type.count("phid")} times, and POE id occur {id_type.count("poe")} times')

From looking at the identifier type we can see that of the 23 requests, 5 were poe so 18 were normal prescription requests. That still leaves 2 requests that are not translated into dispenses. The other reason a request does not become a dispense is when the medication is NULL in *pharmacy*, but populated in *prescriptions*. As a sanity check we can pull in the rows of *pharmacy* with `pharmacy_id` in the medication request. 

In [ ]:
# Grab the pharmacy_ids
pharmacy_ids = [int(fhir['identifier'][0]['value']) for fhir in medreq_p1.fhir \
                if fhir['identifier'][0]['type']['coding'][0]['value'] == 'phid']

# Get pharmacy medication and prescription drug
q_ph_meds = f""" 
    SELECT 
        ph.pharmacy_id
        , ph.medication as ph_medication 
        , pr.drug as pr_medication
    FROM 
        mimic_hosp.pharmacy ph
        LEFT JOIN mimic_hosp.prescriptions pr
            ON ph.pharmacy_id = pr.pharmacy_id
    WHERE ph.pharmacy_id IN {tuple(pharmacy_ids)} 
"""
ph_meds = pd.read_sql_query(q_ph_meds, db_conn)

In [ ]:
ph_meds.sort_values(by='ph_medication')

Two of the pharmacy rows have None values, even though the prescription has a filled in medication. When generating the FHIR resources, we cannot assume that the prescription information gets translated to dispense, so these two None rows get omitted.

Now lets follow one medication for this patient through the full prescribe->dispense->administer process in FHIR. Heparin is a standard medication given regularly out to patients. Let's take the top pharmacy_id and folllow it.

In [ ]:
medication = 'Heparin'
medreq1_id = ph_meds.loc[np.where(ph_meds['ph_medication'] == medication)[0][0]]['pharmacy_id']
medreq1_uuid = str(uuid5(ns_medreq, str(medreq1_id)))

# Medication request timing
medreq = [fhir for fhir in medreq_p1.fhir if fhir['id'] == medreq1_uuid][0]

## Locate all dispense and admin events associated with the medreq
meddisp1_uuids = [fhir['id'] for fhir in meddisp_p1.fhir \
                 if fhir['authorizingPrescription'][0]['reference'] == f'MedicationRequest/{medreq1_uuid}']

medadmin1_uuids = [fhir['id'] for fhir in medadmin_p1.fhir if fhir['request'] != {} 
                  and fhir['request']['reference'] == f'MedicationRequest/{medreq1_uuid}']
#meadmin1_uuids = [fhir for fhir in medadmin_with_requests \
#                 if fhir['request']['reference'] == f'MedicationRequest/{medreq1_uuid}']

# Medication administration info
medadmin1_timing = [datetime.datetime.strptime(fhir['effectiveDateTime'][0:19], '%Y-%m-%dT%H:%M:%S') 
                    for fhir in medadmin_p1.fhir if fhir['id'] in medadmin1_uuids] 

medadmin1_dose = [fhir['dosage']['dose']['value']
                    for fhir in medadmin_p1.fhir if fhir['id'] in medadmin1_uuids] 

medadmin1_dose_unit = [fhir['dosage']['dose']['unit']
                    for fhir in medadmin_p1.fhir if fhir['id'] in medadmin1_uuids] 

In [ ]:
datetime.datetime.strptime('2173-12-03T20:00:00-05:00'[0:19], '%Y-%m-%dT%H:%M:%S')

In [ ]:
medreq['dispenseRequest']['validityPeriod']['start']

In [ ]:
medreq = [fhir for fhir in medreq_p1.fhir if fhir['id'] == medreq1_uuid][0]

# Convert times to datetime, and drop timezone
starttime = datetime.datetime.strptime(medreq['dispenseRequest']['validityPeriod']['start'][0:19], '%Y-%m-%dT%H:%M:%S')
endtime = datetime.datetime.strptime(medreq['dispenseRequest']['validityPeriod']['end'][0:19], '%Y-%m-%dT%H:%M:%S')
dose = int(medreq['dosageInstruction'][0]['doseAndRate'][0]['doseQuantity']['value'])
unit = medreq['dosageInstruction'][0]['doseAndRate'][0]['doseQuantity']['unit']

In [ ]:
starttime

The MedicationRequest information for Heparin is the length of the prescription and the dose prescribed. Below is a visual of the MedicationRequest information for Heparin in this patient.

In [ ]:
# Medication Request
%matplotlib inline
plt.rcParams.update({'font.size': 20})
plt.figure(figsize=[12,8])
x = [starttime, endtime]
y = [dose, dose]
plt.plot(x, y, 'o-', alpha=0.5, label=medication)
plt.ylabel(f'{medication} Dose ({unit})')
plt.xlabel('Datetime')
#plt.xticks(rotation=45)
#plt.legend(loc = "best")
plt.legend(loc = 'lower right')
plt.ylim(bottom=0)
plt.tight_layout()


# concise dates on the x-axis
ax = plt.gca()
locator = mdates.AutoDateLocator(minticks=3, maxticks=7)
formatter = mdates.ConciseDateFormatter(locator)
ax.xaxis.set_major_locator(locator)
ax.xaxis.set_major_formatter(formatter)


plt.show()

The MedicationDispense is next but we do not have a specific time associated with the dispense from MIMIC. The purpose of the MedicationDispense is that the medication can differ going from *prescriptions* to *pharmacy*. Now onto MedicationAdminstration.

The MedicationAdministration information for a patient is the administration time and the dose delivered. Below is a visual represetnation of the MedicationAdministration of heparin in the patient. As you can see the MedicationAdministration timing lines up inside the bounds MedicationRequest time.

In [ ]:
# Medication administration of heparin

#plt.rcParams.update({'font.size': 20})
plt.figure(figsize=[12,8])
x = medadmin1_timing
y = medadmin1_dose
plt.plot(x,y, 'o')
plt.ylabel(f'{medication} Dose ({medadmin1_dose_unit[0]})')
plt.xlabel('Datetime')
plt.ylim(bottom=0)
plt.tight_layout()

# concise dates on the x-axis
ax = plt.gca()
locator = mdates.AutoDateLocator(minticks=3, maxticks=7)
formatter = mdates.ConciseDateFormatter(locator)
ax.xaxis.set_major_locator(locator)
ax.xaxis.set_major_formatter(formatter)

plt.show()

## 5.2 Patient without emar
Prior to 2015 there was no emar records. So all records were manual documentation that are stored in *inputevents* and have been created as MedicationAdministration. No link back to prescriptions is present here as the medication are being given right at the bedside in the ICU. So pre 2015 patients would have no MedicationRequest or MedicationDispense 

In [ ]:
# Medication details
subject_id = 10008100
patient_uuid = str(uuid5(ns_patient, str(subject_id)))

In [ ]:
# Get medadmin_icu associated
q_medadmin_icu = f""" 
    SELECT * 
    FROM mimic_fhir.medication_administration_icu
    WHERE patient_id = '{patient_uuid}' 
"""
medadmin_icu = pd.read_sql_query(q_medadmin_icu, db_conn)

med_ids = [fhir['medicationReference']['reference'].split('/')[1] for fhir in medadmin_icu.fhir]

# Get medication associated with the medadmin events
q_med_icu= f""" 
    SELECT * 
    FROM mimic_fhir.medication
    WHERE id IN {tuple(med_ids)} 
"""
med_icu = pd.read_sql_query(q_med_icu, db_conn)
med_lookup = {fhir['id']: fhir['code']['coding'][0]['code'] for fhir in med_icu.fhir}

Let's look at all the fluids that are given during the patients stay

In [ ]:
medadmin_icu

In [ ]:
fluids = [fhir for fhir in medadmin_icu.fhir if fhir['dosage']['rateQuantity']['unit'] == 'mL/hour']

In [ ]:
medadmin_icu.fhir[3]

In [ ]:
fluids = [fhir for fhir in medadmin_icu.fhir 
          if 'rateQuantity' in fhir['dosage'] and fhir['dosage']['dose']['unit'] == 'ml']
fluids

In [ ]:
# Get fluids with rates
#fluids = [fhir for fhir in medadmin_icu.fhir 
#          if 'rateQuantity' in fhir['dosage'] and fhir['dosage']['rateQuantity']['unit'] == 'ml']

# Get fluids with dose
fluids = [fhir for fhir in medadmin_icu.fhir if fhir['dosage']['dose']['unit'] == 'ml']


# Set colours for fluids
colors = {
    'LR': '#1b9e77',
    'PO Intake': '#9375d1',
    'OR Crystalloid Intake': '#d95f02',
    'Solution': '#325aa8'
    
}


plt.figure(figsize=[12, 12])
for fluid in fluids:
  label = med_lookup[fluid['medicationReference']['reference'].split('/')[1]]
  color = colors[label]
  if 'effectivePeriod' in fluid: #and label == 'Solution':
      x_start = datetime.datetime.strptime(fluid['effectivePeriod']['start'][0:19], '%Y-%m-%dT%H:%M:%S')
      x_end = datetime.datetime.strptime(fluid['effectivePeriod']['end'][0:19], '%Y-%m-%dT%H:%M:%S')
      x = [x_start, x_end]
      dose = pd.to_numeric(fluid['dosage']['dose']['value'])
      plt.plot(x, [dose, dose], 'o-', label=label, color=color, markersize=10)
  elif 'effectiveDateTime' in fluid:
      x_time = datetime.datetime.strptime(fluid['effectiveDateTime'][0:19], '%Y-%m-%dT%H:%M:%S')   
      dose = pd.to_numeric(fluid['dosage']['dose']['value'])
      plt.plot(x_time, dose, 'o', label=label, color=color, markersize=20)


plt.xticks(rotation=45)
plt.ylabel('Fluid Dose (mL)')

# concise dates on the x-axis
ax = plt.gca()
locator = mdates.AutoDateLocator(minticks=3, maxticks=7)
formatter = mdates.ConciseDateFormatter(locator)
ax.xaxis.set_major_locator(locator)
ax.xaxis.set_major_formatter(formatter)

#plt.legend(loc='center right')
# Combine any fluids that are the same
handles, labels = plt.gca().get_legend_handles_labels()
labels, ids = np.unique(labels, return_index=True)
handles = [handles[i] for i in ids]
plt.legend(handles, labels, loc='center right')

plt.show()

In [ ]:
# Get fluids with rates
fluids = [fhir for fhir in medadmin_icu.fhir 
          if 'rateQuantity' in fhir['dosage'] and fhir['dosage']['rateQuantity']['unit'] == 'mL/hour']



# Set colours for fluids
colors = {
    'LR': '#1b9e77',
    'PO Intake': '#9375d1',
    'OR Crystalloid Intake': '#d95f02',
    'Solution': '#325aa8'
    
}


plt.figure(figsize=[12, 12])
for fluid in fluids:
  label = med_lookup[fluid['medicationReference']['reference'].split('/')[1]]
  color = colors[label]
  if 'effectivePeriod' in fluid: #and label == 'Solution':
      x_start = datetime.datetime.strptime(fluid['effectivePeriod']['start'][0:19], '%Y-%m-%dT%H:%M:%S')
      x_end = datetime.datetime.strptime(fluid['effectivePeriod']['end'][0:19], '%Y-%m-%dT%H:%M:%S')
      x = [x_start, x_end]
      dose = pd.to_numeric(fluid['dosage']['rateQuantity']['value'])
      plt.plot(x, [dose, dose], 'o-', label=label, color=color, markersize=10)
  elif 'effectiveDateTime' in fluid:
      x_time = datetime.datetime.strptime(fluid['effectiveDateTime'][0:19], '%Y-%m-%dT%H:%M:%S')   
      dose = pd.to_numeric(fluid['dosage']['rateQuantity']['value'])
      plt.plot(x_time, dose, 'o', label=label, color=color, markersize=20)


plt.xticks(rotation=45)
plt.ylabel('Fluid Dose (mL/hour)')

# concise dates on the x-axis
ax = plt.gca()
locator = mdates.AutoDateLocator(minticks=3, maxticks=7)
formatter = mdates.ConciseDateFormatter(locator)
ax.xaxis.set_major_locator(locator)
ax.xaxis.set_major_formatter(formatter)

#plt.legend(loc='center right')
# Combine any fluids that are the same
handles, labels = plt.gca().get_legend_handles_labels()
labels, ids = np.unique(labels, return_index=True)
handles = [handles[i] for i in ids]
plt.legend(handles, labels, loc='center right')

plt.show()

# <u> Medication identifier work </u>
Look into medication tables in MIMIC: `emar`, `emar_detail`, `input_events`, `prescriptions`, and `pharmacy`

### 1. Medication Identifier
In FHIR, medication is stored as a resource that can be referenced by other medication actions (Request, Dispense, Administration). A common identifier is needed then from MIMIC that can relate across `emar`, `emar_detail`, `input_events`, `prescriptions`, and `pharmacy`.

Potential identifiers
- GSN: generic sequence number for medication, found in `prescriptions`
- NDC: national drug code for medication, found in `prescriptions`
- product_code/product_description: medication details found in `emar_detail`
- drug name: The straight drug name, found in all tables

The final medication identifier could be a single one of the mentioned identifiers but could also be a combination depending on table linking limitations.

### 1.1 Medication Identifier - GSN
GSN is found in `prescriptions`. A typical GSN is a six digit number.

In [ ]:
q_gsn = """ SELECT length(gsn), count(pharmacy_id) FROM mimic_hosp.prescriptions 
            GROUP BY length(gsn) """
gsn = pd.read_sql_query(q_gsn, db_conn)

In [ ]:
# From gsn these should all be 6 digit numerics. The number of NA and varying 
# sizes is concerning for use as an identifier. GSN can have multiple gsn it appears separated by commas
gsn

In [ ]:
# Plot of all gsn lengths, the length of 6 is clearly the majority
plt.bar(gsn['length'], gsn['count'], width=6)

In [ ]:
# Excluding the 6 digit GSNs (the majority), we can see empty gsn are found
# Not seen in the graph also are the ~2 million NA values for GSN
plt.bar(gsn['length'].drop(1, axis=0), gsn['count'].drop(1, axis=0), width =6)

**Conclusion**: GSN has too much variance and missing data to be used as the medication identifier

### 1.2 Medication Identifier - NDC
The NDC is found in `prescriptions` and should be a 11 digit numeric identifier

In [ ]:
q_ndc = """ SELECT length(ndc), count(pharmacy_id) FROM mimic_hosp.prescriptions 
            GROUP BY length(ndc) """
ndc = pd.read_sql_query(q_ndc, db_conn)
ndc

NDC values with length zero are primarly just zero values, so missing data. NDC unfortunately is missing ~2 million medication codes

**Conclusion**: NDC is missing too may values to be the medication identifier, but is a good target for medication codes in future concept mapping

### 1.3 Medication Identifier - Product Code
The product code is found in `emar_detail`. The one limitate of the product code is that there is no linkage to the ICU tables. Could potentially link back using just the drug name

In [ ]:
q_product_code = """ 
    SELECT 
        SUM(CASE WHEN product_code IS NULL THEN 1 ELSE 0 END) AS null_count
        , SUM(CASE WHEN product_code IS NOT NULL THEN 1 ELSE 0 END) AS valid_count
    FROM mimic_hosp.emar_detail
"""
product_code = pd.read_sql_query(q_product_code, db_conn)
product_code

**Conclusion**: Again way too many null values(~29 million) for this to be the sole identifier for medication

### 1.4 Medication Identifier - Drug Name
The drug name is common across all the medication tables in MIMIC. The main limitation is that product specific information would be missing if the drug name is used alone.

In [ ]:
# Medication from prescriptions (10,255 distinct values)
q_pr_meds = "SELECT drug as medication FROM mimic_hosp.prescriptions"
pr_meds = pd.read_sql_query(q_pr_meds, db_conn)
pr_meds.medication.unique().size

In [ ]:
# Medication from pharmacy (10,229 distinct values)
q_ph_meds = "SELECT medication FROM mimic_hosp.pharmacy"
ph_meds = pd.read_sql_query(q_ph_meds, db_conn)
ph_meds.medication.unique().size

In [ ]:
# Medication from emar (4,293 distinct values)
q_em_meds = "SELECT medication FROM mimic_hosp.emar"
em_meds = pd.read_sql_query(q_em_meds, db_conn)
em_meds.medication.unique().size

In [ ]:
# Medication from inputevents/d_items (470 distinct values)
q_ie_meds = """
    SELECT di.label as medication
    FROM mimic_icu.d_items di
    WHERE di.linksto = 'inputevents'
"""
ie_meds = pd.read_sql_query(q_ie_meds, db_conn)
ie_meds.medication.unique().size

In [ ]:
# Check for NA values 
data = {'Tables' : ['prescriptions', 'pharmacy', 'emar', 'inputevents'],
        'NA Count' : [pr_meds.isna().sum().medication, 
                      ph_meds.isna().sum().medication, 
                      em_meds.isna().sum().medication, 
                      ie_meds.isna().sum().medication]}
print(pd.DataFrame(data))

So pharmacy and emar both have some NA values, lets investigate the scale and reason behind that
Primary reason is likely IV meds

In [ ]:
q_emar = """
    SELECT 
        em.medication
        , em.pharmacy_id as em_pharmacy_id
        , ed.pharmacy_id as ed_pharmacy_id
        --, ed.parent_field_ordinal
        , ed.product_code
    FROM 
        mimic_hosp.emar em
        LEFT JOIN mimic_hosp.emar_detail ed 
            ON em.emar_id = ed.emar_id 
    WHERE
        em.medication IS NULL
        AND ed.parent_field_ordinal IS NOT NULL
"""
emar = pd.read_sql_query(q_emar, db_conn)
# values still have pharmacy_ids which link back to actual values (mimic update to fill these values?)

In [ ]:
print(f'Null medication in emar size: {emar.size}')
emar

In [ ]:
# Most of the remaining values have a pharmacy_id or product code to link to, 
# but there is still a subset missing that too!

emar.loc[(emar.medication.isnull()) 
         & (emar.em_pharmacy_id.isnull()) 
         & (emar.ed_pharmacy_id.isnull()) 
         & (emar.product_code.isnull()) 
        ].info()

# These emar have no information for FHIR, so we will need to filter 
# out the ~50,000 emar events with no related medication, potentially IV but no link to it (maybe poe)

In [ ]:
# Check pharmacy table null values
q_pharma = """
    SELECT *
    FROM 
        mimic_hosp.pharmacy ph
    WHERE
        ph.medication IS NULL
"""
pharma = pd.read_sql_query(q_pharma, db_conn)
pharma.info()

The `proc_type` is indicative what kind of medication is being delivered. Since the `proc_type` is non-null for all values where medication is null we can glean the medication intent.

In [ ]:
pharma.proc_type.unique()

In [ ]:
pharma.groupby(['proc_type']).size()

The primary offender is IV/TPN, so we can decide if these just get mapped to one thing. The few irrigation/unit dose cases could probably just be omitted.

### 1.5 Medication Identifier - Decisions
From the options the best option initially will be to use medication names. Limitations with using names:
- There are ~55,000 emar events without an explicit medication name or pharmacy link. Likely IV meds
- There are ~1 million pharmacy entries without an explicit medication name. Primarily IV procedures, so again could be grouped into IV meds
- The `prescriptions` table can have multiple drugs under the same pharmacy_id, so they will need to be grouped
  - Proposal for now is to use the format MAIN_BASE_ADDITIVE based on drug_type to concatenate values 
  - Other option is to concatenate them alphabetically

## 2. Medication Examples
Look into specific cases for medication
- Pills - ranitidine/acetominophin
- Infusion - heparin/noepinephrine
- Antibiotic - vancamycin
- Saline - IV entries

Grab the medication administrations from one patient, who took them all throughout a hopsital stay

In [ ]:
subject_id = 10012853
q_med_ex = f"""
    SELECT 
        em.*
        , ed.emar_seq
        , ed.parent_field_ordinal
        , ed.administration_type
        , ed.barcode_type
        , ed.dose_due
        , ed.dose_due_unit
        , ed.dose_given
        , ed.dose_given_unit
        , ed.product_amount_given
        , ed.product_unit
        , ed.product_description
        , ed.product_code
        , ed.infusion_rate
        , ed.infusion_rate_unit
        , ed.route
    FROM 
        mimic_hosp.emar em
        LEFT JOIN mimic_hosp.emar_detail ed
            ON em.emar_id = ed.emar_id    
    WHERE em.subject_id = {subject_id}
"""
med_ex = pd.read_sql_query(q_med_ex, db_conn)


And you can see the different medications and the count of taking them

In [ ]:
med_ex.groupby(['medication']).size()

### 2.1 Medication Examples - Pills

In [ ]:
pill = 'Acetaminophen' # Aspirin

In [ ]:
idx = med_ex['parent_field_ordinal'].notnull()

In [ ]:
emar = med_ex.loc[idx, :].copy()

In [ ]:
pills = emar[(emar.medication == pill)]

In [ ]:
# The typical product_unit for something like Acetaminophen is a tablet (TAB)
pills.product_unit

cols = ['subject_id', 'emar_id', 'pharmacy_id', 'charttime', 'medication',
       'administration_type', 'dose_given', 'dose_given_unit', 'product_unit']

pills[cols]

### 2.2 Medication Examples - Infusion
Look into heparin in emar and inputevents

### 2.2.1 - Infusion emar

In [ ]:
infusion = 'Heparin'

In [ ]:
infusions = med_ex[(med_ex.medication == infusion)]

In [ ]:
cols = ['subject_id', 'emar_id', 'pharmacy_id', 'charttime', 'medication',
       'administration_type', 'dose_given', 'dose_given_unit', 'product_unit',
       'infusion_rate', 'infusion_rate_unit']

infusions[cols].head(n=10)

In [ ]:
infusions.groupby(['medication']).size()

In [ ]:
q_heparin = """
    SELECT * 
    FROM 
        mimic_hosp.emar em
        LEFT JOIN mimic_hosp.emar_detail ed
            ON em.emar_id = ed.emar_id   
    WHERE medication = 'Heparin'
    LIMIT 1000
"""
heparin = pd.read_sql_query(q_heparin, db_conn)

In [ ]:
heparin.info()

In [ ]:
heparin[0:10]

- Heparin is delivered with infusion, but the 

### 2.2.2 infusion inputevents